<a href="https://colab.research.google.com/github/aledelorbe/pt2Codigos/blob/main/SegundasActividades/Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cargar librerias

In [ ]:
# VERSION 1
# X1 = df2010[['ent_resid_nm', 'Organ']]
# one_hot_encoder = OneHotEncoder(sparse_output=False)

# # Ajustar y transformar las columnas seleccionadas
# X1_encoded = one_hot_encoder.fit_transform(X1)


# VERSION 2. Manteien las columnas originales y les pone un nombre a las
# columnas
# # Obtener las columnas categóricas
# columns_to_encode = df2010.columns[:2]

# # Realizar la codificación one-hot de esas columnas
# X1 = pd.get_dummies(df2010, columns=columns_to_encode)

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

### Cargar los dataFrame

In [24]:
nuevaMortalidadLimpia = '/content/drive/MyDrive/Pt2/Primeras Actividades/0 Bases de Datos/Nueva Mortalidad Limpia/'

df2010 = pd.read_csv(nuevaMortalidadLimpia + '2010.csv')
df2011 = pd.read_csv(nuevaMortalidadLimpia + '2011.csv')
df2012 = pd.read_csv(nuevaMortalidadLimpia + '2012.csv')
df2013 = pd.read_csv(nuevaMortalidadLimpia + '2013.csv')
df2014 = pd.read_csv(nuevaMortalidadLimpia + '2014.csv')
df2015 = pd.read_csv(nuevaMortalidadLimpia + '2015.csv')
df2016 = pd.read_csv(nuevaMortalidadLimpia + '2016.csv')
df2017 = pd.read_csv(nuevaMortalidadLimpia + '2017.csv')
df2018 = pd.read_csv(nuevaMortalidadLimpia + '2018.csv')
df2019 = pd.read_csv(nuevaMortalidadLimpia + '2019.csv')

In [25]:
df2010

,ent_resid_nm,Organ,desc_ocupacion,desc_escolaridad
0,Aguascalientes,Pancreas,"Trabajadores en actividades agrícolas, ganader...","Funcionarios, directores y jefes"
1,Aguascalientes,Bladder,"Trabajadores en actividades agrícolas, ganader...","Funcionarios, directores y jefes"
2,Aguascalientes,Lungs,"Trabajadores en actividades agrícolas, ganader...","Funcionarios, directores y jefes"
3,Jalisco,Lungs,"Trabajadores en actividades agrícolas, ganader...","Funcionarios, directores y jefes"
4,Aguascalientes,Kidney,"Trabajadores en actividades agrícolas, ganader...","Funcionarios, directores y jefes"
...,...,...,...,...
64292,Zacatecas,Ovary,No especificada,Trabajadores en actividades elementales y de a...
64293,Sinaloa,Pancreas,No especificada,Trabajadores en actividades elementales y de a...
64294,Tabasco,Mylenoma,Trabajadores artesanales,Trabajadores en actividades elementales y de a...
64295,Puebla,Pancreas,Trabajadores en actividades elementales y de a...,Trabajadores en actividades elementales y de a...




### Transformaciones

In [26]:
X1 = df2010[['ent_resid_nm', 'Organ']]
# X1 = df2010.iloc[:, [0, 1]].values

In [27]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1])], remainder='passthrough')
X1 = np.array(ct.fit_transform(X1)).astype('float64')
X1

ValueError: ignored

In [21]:
print(X1)

  (0, 0)	1.0
  (0, 58)	1.0
  (1, 0)	1.0
  (1, 32)	1.0
  (2, 0)	1.0
  (2, 48)	1.0
  (3, 13)	1.0
  (3, 48)	1.0
  (4, 0)	1.0
  (4, 44)	1.0
  (5, 0)	1.0
  (5, 48)	1.0
  (6, 0)	1.0
  (6, 59)	1.0
  (7, 0)	1.0
  (7, 38)	1.0
  (8, 0)	1.0
  (8, 58)	1.0
  (9, 0)	1.0
  (9, 59)	1.0
  (10, 31)	1.0
  (10, 46)	1.0
  (11, 13)	1.0
  (11, 48)	1.0
  (12, 0)	1.0
  :	:
  (64284, 59)	1.0
  (64285, 29)	1.0
  (64285, 32)	1.0
  (64286, 29)	1.0
  (64286, 46)	1.0
  (64287, 29)	1.0
  (64287, 48)	1.0
  (64288, 29)	1.0
  (64288, 59)	1.0
  (64289, 29)	1.0
  (64289, 34)	1.0
  (64290, 29)	1.0
  (64290, 43)	1.0
  (64291, 30)	1.0
  (64291, 60)	1.0
  (64292, 31)	1.0
  (64292, 57)	1.0
  (64293, 24)	1.0
  (64293, 58)	1.0
  (64294, 26)	1.0
  (64294, 53)	1.0
  (64295, 20)	1.0
  (64295, 58)	1.0
  (64296, 26)	1.0
  (64296, 64)	1.0


In [22]:
print(X1[:3])

IndexError: ignored

In [13]:
# Todas las variables son NO ordinales

#varOneHotEnco = ColumnTransformer( [('one_hot_encoder', OneHotEncoder(categories = 'auto'), [0, 1])], remainder = 'passthrough')

#X1 = np.array(varOneHotEnco.fit_transform(X1))


In [14]:
print(X1)


  (0, 0)	1.0
  (0, 58)	1.0
  (1, 0)	1.0
  (1, 32)	1.0
  (2, 0)	1.0
  (2, 48)	1.0
  (3, 13)	1.0
  (3, 48)	1.0
  (4, 0)	1.0
  (4, 44)	1.0
  (5, 0)	1.0
  (5, 48)	1.0
  (6, 0)	1.0
  (6, 59)	1.0
  (7, 0)	1.0
  (7, 38)	1.0
  (8, 0)	1.0
  (8, 58)	1.0
  (9, 0)	1.0
  (9, 59)	1.0
  (10, 31)	1.0
  (10, 46)	1.0
  (11, 13)	1.0
  (11, 48)	1.0
  (12, 0)	1.0
  :	:
  (64284, 59)	1.0
  (64285, 29)	1.0
  (64285, 32)	1.0
  (64286, 29)	1.0
  (64286, 46)	1.0
  (64287, 29)	1.0
  (64287, 48)	1.0
  (64288, 29)	1.0
  (64288, 59)	1.0
  (64289, 29)	1.0
  (64289, 34)	1.0
  (64290, 29)	1.0
  (64290, 43)	1.0
  (64291, 30)	1.0
  (64291, 60)	1.0
  (64292, 31)	1.0
  (64292, 57)	1.0
  (64293, 24)	1.0
  (64293, 58)	1.0
  (64294, 26)	1.0
  (64294, 53)	1.0
  (64295, 20)	1.0
  (64295, 58)	1.0
  (64296, 26)	1.0
  (64296, 64)	1.0


In [ ]:
print(df2010[1:5])

     ent_resid_nm                                         detailCaus  \
1  Aguascalientes  Tumor maligno de la vejiga urinaria, parte no ...   
2  Aguascalientes  Tumor maligno de los bronquios o del pulmón, p...   
3         Jalisco  Tumor maligno de los bronquios o del pulmón, p...   
4  Aguascalientes  Tumor maligno del riñón, excepto de la pelvis ...   

                                      desc_ocupacion  \
1  Trabajadores en actividades agrícolas, ganader...   
2  Trabajadores en actividades agrícolas, ganader...   
3  Trabajadores en actividades agrícolas, ganader...   
4  Trabajadores en actividades agrícolas, ganader...   

                   desc_escolaridad  
1  Funcionarios, directores y jefes  
2  Funcionarios, directores y jefes  
3  Funcionarios, directores y jefes  
4  Funcionarios, directores y jefes  


In [ ]:
print(type(df2010))
print(df2010.dtype)
# print(df2010)

<class 'pandas.core.frame.DataFrame'>


AttributeError: ignored

In [ ]:
df2010 = df2010.astype(np.float64)